In [6]:
reload_lamb()

In [23]:
lamb.meta.core.TypedExpr._parsing_locals

{'__ptf': <function lamb.meta.core.__ptf(s, typ=None, assignment=None)>,
 'Disjunctive': <function lamb.meta.core.TypedExpr.add_local.<locals>.<lambda>(*args, **kwargs)>,
 'Fun': <function lamb.meta.core.TypedExpr.add_local.<locals>.<lambda>(*args, **kwargs)>,
 'Partial': <function lamb.meta.core.OperatorRegistry.op_instantiator.<locals>.<lambda>(seq, **kwargs)>,
 'Body': <function lamb.meta.core.OperatorRegistry.op_instantiator.<locals>.<lambda>(seq, **kwargs)>,
 'Condition': <function lamb.meta.core.OperatorRegistry.op_instantiator.<locals>.<lambda>(seq, **kwargs)>,
 'Case': <function lamb.meta.core.OperatorRegistry.op_instantiator.<locals>.<lambda>(seq, **kwargs)>,
 'Dom': <function lamb.meta.core.OperatorRegistry.op_instantiator.<locals>.<lambda>(seq, **kwargs)>,
 'Codom': <function lamb.meta.core.OperatorRegistry.op_instantiator.<locals>.<lambda>(seq, **kwargs)>}

In [8]:
x = %te Case(p_t & q_t, _c1, _c2)
x

Case((p_t & q_t), _c1, _c2)

In [9]:
x.simplify_all()

Case((p_t & q_t), _c1, _c2)

In [10]:
repr(x)

'Case((p_t & q_t), _c1, _c2)'

In [11]:
x2 = %te Case(p_t & q_t, _c1, Case(r_t, _c2, _c3))
x2

Case((p_t & q_t), _c1, Case(r_t, _c2, _c3))

In [12]:
repr(x2)

'Case((p_t & q_t), _c1, Case(r_t, _c2, _c3))'

In [13]:
x2 = %te Case(p_t & q_t, Case(s_t, _c1, _c4), Case(r_t, _c2, _c3))
x2

Case((p_t & q_t), Case(s_t, _c1, _c4), Case(r_t, _c2, _c3))

In [14]:
x3 = %te Case(p_t | ~p_t, _c1, _c2)
x3

Case((p_t | ~p_t), _c1, _c2)

In [15]:
x3.simplify_all().derivation

1. Case((p_t | ~p_t), _c1, _c2)    ([excluded middle])
 2. Case(True, _c1, _c2)    (Case simplification: True)
 3. _c1

In [16]:
x4 = %te Case(p_t & ~p_t, _c1, Case(p_t | ~p_t, _c2, _c3))
x4

Case((p_t & ~p_t), _c1, Case((p_t | ~p_t), _c2, _c3))

In [17]:
x4.simplify_all().derivation.trace()

DisplayNode instance: HTML rendering only

In [18]:
reload_lamb()

In [19]:
x5 = %te Case(p_t, _c1, Case(q_t, _c2, _c3))
x5

Case(p_t, _c1, Case(q_t, _c2, _c3))

In [20]:
lamb.meta.exec(x5, p=False, q=False)

'_c3'

In [21]:
x5._compiled({'p':True})

'_c1'

In [22]:
lamb.meta.exec(x5, p=True)

TypeError: Context for executing `Case(p_t, _c1, Case(q_t, _c2, _c3))` is missing free term(s): q_t

In [1]:
reload_lamb()

In [39]:
from lamb.meta import sets, quantifiers
from lamb.meta import TypedExpr, derived
from lamb.meta.ply import get_sopt
from lamb.meta.boolean import true_term
from lamb.meta.quantifiers import RestrictedBindingOp

class UnionOp(quantifiers.RestrictedBindingOp):
    canonical_name = "Union"
    op_name_uni = "∪"
    op_name_latex = "\\bigcup{}"
    pre_simplify = True

    def __init__(self, var, body, /, restrictor=None, typ=None, **kwargs):
        self.type_constraint(body.type, types.SetType, constant=True)
        body, typ = self.type_constraint(body, typ,
                            error=f"`UnionOp` body and type should match")
    
        super().__init__(var, body, restrictor=restrictor, typ=body.type, **kwargs)

    def to_exists_form(self):
        outer_var = meta.term(self.find_safe_variable(), typ=self.type.content_type)
        inner_var = self.var_instance.copy()
        new_body = outer_var << self.body
        result = sets.ConditionSet(outer_var,
                                   quantifiers.Exists(inner_var, new_body, self.restrictor))        
        return derived(result, self, f"∪{self.varname} => ∃")
    
    def eliminate(self, assignment=None, **sopts):
        if self.vacuous():
            # covers empty set / domain set as body
            return derived(self.body.copy(), self, f"trivial ∪{self.varname} elimination")
        elif self.finite_safe():
            a = self.scope_assignment(assignment=assignment)
            subs = [self[1].under_assignment(a | {self.varname : elem})
                    for elem in self.domain_iter()]
            return derived(sets.SetUnion.join(subs, empty=true_term),
                self,
                f"∪{self.varname} => ∪")
        # disable this for now -- it produces more complex formulas, not simpler ones
        # else:
        #     return self.to_exists_form()

        return self

    def simplify(self, assignment=None, **sopts):
        if (self.vacuous() or self.domain_cardinality() == 1
                    or get_sopt('eliminate_sets', sopts)):
            return self.eliminate(assignment=assignment, **sopts)
        return self

lamb.meta.core.registry.add_binding_op(UnionOp)

WARNING (core): Overriding existing binding operator `Union` in registry


In [40]:
test = %te Union x_e << Human_{e}: {x}
test

(Union x_e << Human_{e}: {x_e})

In [41]:
test.to_exists_form().simplify_all()

(Set x1_e: (Exists x_e << Human_{e}: (x1_e <=> x_e)))

In [42]:
test.eliminate().simplify_all()

(Union x_e << Human_{e}: {x_e})

In [43]:
test.simplify_all()

(Union x_e << Human_{e}: {x_e})

In [44]:
test2 = %te Union x_e << {_c1, _c2, _c3}: {x}
test2

(Union x_e << {_c2, _c3, _c1}: {x_e})

In [45]:
test2.simplify_all(eliminate_sets=True)

{_c2, _c3, _c1}